## DecisionTreeClassifier and others from SAS® Viya® on Heart Disease
### Source
This example is adapted from [Heart Disease prediction Random forest Classifier](https://www.kaggle.com/code/mruanova/heart-disease-prediction-random-forest-classifier) by Mau Rua.

### Data Preparation
#### About the data set
The original data contains 76 different attributes of patients from four different hospital databases.  The goal is to determine if the attributes can be used to predict whether patients are diagnosed with heart disease.  However, this data has been subset to contain only 14 factors from only the Cleveland database.  The variables included and their interpretations are:
- age
- trestbps: resting blood pressure
- chol: serum cholesterol
- thalch: maximum heart rate achieved
- ca: number of major vessels (0-3) colored by flourosopy
- sex
- cp: chest pain type
- exang: exercise-induced angina
- slope: slope of the peak exercise ST segment
- thal:  thalassemia (3 = normal; 6 = fixed defect; 7 = reversable defect)
- restecg: resting electrocardiographic results
- fbs: fasting blood sugar
- target: diagnosis of heart disease 
- oldpeak: ST depression induced by exercise relative to rest

#### Importing the data set

In [ ]:
import os
import pandas as pd
import numpy as np
import warnings

from sasviya.ml.linear_model import LogisticRegression
from sasviya.ml.tree import DecisionTreeClassifier, ForestClassifier, GradientBoostingClassifier

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

workspace=f'{os.path.abspath("")}/../data/'
heart_df=pd.read_csv(workspace+'heart_disease.csv')
heart_df.shape

In [ ]:
heart_df.head(3)

### Data Preprocessing
We will start by getting some general characteristics about the data set.

In [ ]:
heart_df.info()

#### Replacing NaN values with mean
In examining the data, we see there are some missing values.  We will replace those with the mean for the column.

In [ ]:
cols_sum_null = heart_df.isnull().sum()
print('Missing value counts (before imputation)\n', cols_sum_null)

In [ ]:
hasnull_cols = cols_sum_null[cols_sum_null != 0]
for col in hasnull_cols.index:
    mean = heart_df[col].mean()
    heart_df[col].fillna(mean, inplace=True)
print('Missing value counts (after imputation)\n', heart_df.isnull().sum())

### Visualizing the data
To help understand the data, we will generate some graphs and perform some additional analysis.
- a histogram of target for "Heart Disease" versus "No disease"
- a pie chart visualization of the same information
- a pairplot of quantitative features
- a heatmap of feature correlations

In [ ]:
disease = len(heart_df[heart_df['target'] == 1])
no_disease = len(heart_df[heart_df['target']== 0])

x = [disease, no_disease]
y = ['Heart Disease', 'No Disease']

In [ ]:
fig, ax = plt.subplots(figsize=(6, 2))
y_pos = np.arange(len(y))
ax.barh(y_pos, x, align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(y)
ax.invert_yaxis() # labels read top-to-bottom
ax.set_xlabel('Count')
ax.set_title('Target')
for i, v in enumerate(x):
    ax.text(v, i, str(v), color='black', va='center', fontweight='normal')

In [ ]:
fig1, ax1 = plt.subplots(figsize=(4, 4))
ax1.pie(x,  labels=y, autopct='%1.1f%%', startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.title('Target categories', size=16)

In [ ]:
#identify quantitative (continuous) features based on number of unique categories
cat_threshold = 8
qualitative = [c for c in heart_df.columns if len(heart_df[c].unique()) <= cat_threshold]
quantitative = [c for c in heart_df.columns if len(heart_df[c].unique()) > cat_threshold]

In [ ]:
print('Qualitative features:\n', qualitative)

In [ ]:
print('Quantitative features:\n', quantitative)

In [ ]:
sn.pairplot(heart_df[quantitative])

In [ ]:
corr = heart_df.corr()
f,ax = plt.subplots(figsize=(8, 8))
hm = sn.heatmap(corr, square=True, ax=ax, annot=True, cmap='Pastel1_r', fmt='.2f', annot_kws={'size':8})
plt.title('Feature Correlations', size=16)

#### Creating training and test data
We split the original data by putting 80% into the training set and 20% into the test set.  We will also examine the shape of the full data and the two new subsets.

In [ ]:
X = heart_df.drop('target', axis=1)
y = heart_df['target']
print('Full data')
print('Shape of X and y respectively :', X.shape, y.shape)

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print('After split')
print('Shape of X and y respectively (train) :', X_train.shape, y_train.shape)
print('Shape of X and y respectively (test) :', X_test.shape, y_test.shape)

In [ ]:
X_train.head()

In [ ]:
X_test.head()

### Building and Training Models with Four Different Alogorithms
We will train four different models against the training set: LogisticRegression, DecisionTreeClassifier, ForestClassifier, and GradientBoostingClassifier.  In each case, the following will also be examined for each run:
- Model accuracy
- Confusion matrix
- ROC plot

In [ ]:
# Common model fitting and assessment code

def fit_model(X_train, y_train, X_test, y_test, model):
    print(model.name.split('_')[0], 'Accuracy\n')
    model.fit(X_train, y_train)
    y_pred_test = model.predict(X_test)

    train_score =  model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)

    print(f'Training accuracy: {train_score:.2f}')
    print(f'Test accuracy: {test_score:.2f}')

    return y_pred_test, test_score


def accuracy_report(y_test, y_pred_test, model):
    output = pd.DataFrame({'Predicted':y_pred_test}) # Heart-Disease yes or no? 1/0
    people = output.loc[output.Predicted == 1]["Predicted"]
    rate_people = 0
    if len(people) > 0 :
        rate_people = len(people)/len(output)
    print(f"% of people predicted with heart-disease: {rate_people*100:.2f}\n")
    print('Classification Report for', model.name.split('_')[0], '\n', classification_report(y_test,y_pred_test))


def confusion_plot(y_test, y_pred_test, model):
    confusion_results = confusion_matrix(y_test,y_pred_test)
    class_names = [0,1]
    fig,ax = plt.subplots() #
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks,class_names)
    plt.yticks(tick_marks,class_names)
    sn.heatmap(pd.DataFrame(confusion_results), annot = True, cmap = 'Pastel1_r', fmt = 'g')
    ax.xaxis.set_label_position('top')
    plt.tight_layout()
    plt.title('Confusion Matrix for ' + model.name.split('_')[0])
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')

def roc_plot(X_test, y_test, model):
    probs = model.predict_proba(X_test).to_numpy()
    preds = probs[:,1]
    fpr, tpr, threshold = roc_curve(y_test, preds)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, label='Model (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC for ' + model.name.split('_')[0])
    plt.legend(loc="lower right")


#### LogisticRegression

For details about using the `LogisticRegression` class in the `sasviya` package, see the [LogisticRegression documentation](https://documentation.sas.com/?cdcId=workbenchcdc&cdcVersion=default&docsetId=explore&docsetTarget=n0110bswc89wqjn1tht4ceu4hs7y.htm).

In [ ]:
logreg = LogisticRegression(
        solver='lbfgs',
        tol=1e-4,
        max_iter=1000)

y_pred_test, score_logreg = fit_model(X_train, y_train, X_test, y_test, model=logreg)
accuracy_report(y_test, y_pred_test, model=logreg)
confusion_plot(y_test, y_pred_test, model=logreg)
roc_plot(X_test, y_test, model=logreg)

### DecisionTreeClassifier

For details about using the `DecisionTreeClassifier` class in the `sasviya` package, see the [DecisionTreeClassifier documentation](https://documentation.sas.com/?cdcId=workbenchcdc&cdcVersion=default&docsetId=explore&docsetTarget=p14rqs4yfhf5bcn1js9nlfgzx795.htm).

In [ ]:
dt = DecisionTreeClassifier(max_depth=5)
y_pred_test, score_dtc = fit_model(X_train, y_train, X_test, y_test, model=dt)
accuracy_report(y_test, y_pred_test, model=dt)
confusion_plot(y_test, y_pred_test, model=dt)
roc_plot(X_test, y_test, model=dt)

#### ForestClassifier

For details about using the `ForestClassifier` class in the `sasviya` package, see the [ForestClassifier documentation](https://documentation.sas.com/?cdcId=workbenchcdc&cdcVersion=default&docsetId=explore&docsetTarget=p04zhxjh60eutqn1t40f0104gw42.htm).

In [ ]:
fc = ForestClassifier(
    n_estimators=100,
    max_depth=5,
    min_samples_leaf=1,
    max_features=None,
    criterion='gini',
    random_state=1
)

y_pred_test, score_fc = fit_model(X_train, y_train, X_test, y_test, model=fc)
accuracy_report(y_test, y_pred_test, model=fc)
confusion_plot(y_test, y_pred_test, model=fc)
roc_plot(X_test, y_test, model=fc)


#### GradientBoostingClassifier

For details about using the `GradientBoostingClassifier` class in the `sasviya` package, see the [GradientBoostingClassifier documentation](https://documentation.sas.com/?cdcId=workbenchcdc&cdcVersion=default&docsetId=explore&docsetTarget=n1kiea90s0276wn1xr0ig0hvkix6.htm).

In [ ]:
gb = GradientBoostingClassifier(n_estimators=100,
                                max_depth=5,
                                min_samples_leaf=1,
                                max_features=None,
                                learning_rate = 0.1,
                                subsample = 1.0,
                                random_state=1)

y_pred_test, score_gbc = fit_model(X_train, y_train, X_test, y_test, model=gb)
accuracy_report(y_test, y_pred_test, model=gb)
confusion_plot(y_test, y_pred_test, model=gb)
roc_plot(X_test, y_test, model=gb)

### Summarizing the results
Using a histogram, we will examine the accuracy scores of each model and identify which performed best on this training and test set.

In [ ]:
algorithms = ['LogisticRegression', 'GradientBoosting', 'Forest', 'DecisionTree']
scores = [round(s,2) for s in [score_logreg, score_gbc, score_fc, score_dtc]]

scores_df = pd.DataFrame(zip(algorithms, scores))
scores_df.columns = ['Name','Test Accuracy']

scores_df.sort_values(by='Test Accuracy', inplace=True, ascending = False)

In [ ]:
plt.rcdefaults()
fig, ax = plt.subplots()

y_pos = np.arange(len(scores_df)) # scores
bars = ax.barh(y_pos, scores_df['Test Accuracy'], align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(scores_df['Name'])
ax.invert_yaxis() # labels read top-to-bottom
ax.set_xlabel('Test Accuracy')
ax.set_title('Which algorithm is best?')
ax.bar_label(bars)